We can't just stop at game data. That's why I also wanted to collect. Player Data and team data.

In [ ]:
import re

# webpage = open('../GIT_NO/NBA Players List (2023-2024).htm').read()

# players = re.findall(r'<a [\s\S]*? href="(/basketball/player/.*/.*)">', webpage)

# players = list(set(players))
# for i in range(len(players)):
#     players[i] = players[i].split('"')[0]
    
# players = list(map(lambda x: '' + x + "/totals", players))
# players    


In [ ]:
# with open('player_links.txt', 'w') as f:
#     for line in players:
#         f.write("%s\n" % line)

## Scraping

Now to take the data out of the webpages

For the player webpages it's a lot more of a simple design so we should be able to take everything out with one regex looking for \<div class="table__inner"> 

In [ ]:
import urllib.request

url = "[redacted]"

fp = urllib.request.urlopen(url)
mybytes = fp.read()

webpage = mybytes.decode("utf8")
fp.close()

# print(webpage)

data = re.findall(r'(<div class="table__inner">[\s\S]*?</main>)', webpage)
data

In [ ]:
Nba = re.findall(r'(<table class="table">[\s\S]*?</table>)', data[0])
Nba_stats = Nba[3]

In [ ]:
player_headers = re.findall(r'(<th class=".*">[\s\S]*?</th>)', Nba_stats)
player_headers = list(map(lambda x: x.split('>')[1].split('<')[0], player_headers))
player_headers

In [ ]:
player_data = re.findall(r'(<td class=".*">[\s\S]*?</td>)', Nba_stats)
player_data = [re.sub(r'(<td class=\".*\">|<\/td>|<a [\s\S]*?>|<\/a>|<div [\s\S]*?>[\s\S]*?>[\s\S]*?<\/div>|\n)','', data) for data in player_data]
player_data = [re.sub(r'[\s]+', '', data) for data in player_data]
player_data

In [ ]:
player_obj = {player_headers[i]: [] for i in range(len(player_headers))}

row_iterator = 0
for i in range(len(player_data)):
    if row_iterator == len(player_headers):
        row_iterator = 0
        player_obj[player_headers[row_iterator]].append(player_data[i])
        row_iterator += 1
    else:
        player_obj[player_headers[row_iterator]].append(player_data[i])
        row_iterator += 1
        
player_obj['Pts'] = player_obj['Pts'][0:len(player_obj['Pts']):2]
player_obj['Reb'] = player_obj['Reb'][0:len(player_obj['Reb']):2]
player_obj['Ast'] = player_obj['Ast'][0:len(player_obj['Ast']):2]

player_obj

Now we're going to convert the data types. Some these are Strings like W-L which could be better represented by a numpy array of size 2. Even the 2P column could be represented by it's floating point value to be more interpretable by machines.

In [ ]:
import pandas as pd
import numpy as np
player_df = pd.DataFrame(player_obj)

for index, row in player_df.iterrows():
    player_df.at[index, 'W-L'] = np.array(player_df.at[index, 'W-L'].split('-'), dtype=int)
    player_df.at[index, '2P'] = (int(player_df.at[index, '2P'].split('/')[0]) / int(player_df.at[index, '2P'].split('/')[1])) 
    # repeat for 3P, FT, FG 
    player_df.at[index, '3P'] = (int(player_df.at[index, '3P'].split('/')[0]) / int(player_df.at[index, '3P'].split('/')[1]))
    player_df.at[index, 'FT'] = (int(player_df.at[index, 'FT'].split('/')[0]) / int(player_df.at[index, 'FT'].split('/')[1]))
    player_df.at[index, 'FG'] = (int(player_df.at[index, 'FG'].split('/')[0]) / int(player_df.at[index, 'FG'].split('/')[1]))

player_df

## Team data

Rinse and repeat for the teams